# Data Prepairing

In [1]:
!pip install datasets evaluate --upgrade -q
!pip install spacy --upgrade -q
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 30.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import random
import numpy as np

import spacy
import datasets
from torchtext import vocab

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import tqdm

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [3]:
dataset = datasets.load_dataset("bentrevett/multi30k")

train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from functools import partial

def tokenize_and_lower(s, key):
    global en_nlp, de_nlp, max_length

    if key == 'en':
        nlp = en_nlp
        text = s['en']
    elif key == 'de':
        nlp = de_nlp
        text = s['de']
    else:
        raise ValueError("Invalid key. Expected 'en' or 'de'.")

    return [token.text.lower() for token in nlp.tokenizer(text)][:max_length]

tokenize_en = partial(tokenize_and_lower, key='en')
tokenize_de = partial(tokenize_and_lower, key='de')

en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

min_freq = 2
max_length = 100
convert_to_lowercase = True
sos_token = "<sos>"
eos_token = "<eos>"
unk_token = "<unk>"
pad_token = "<pad>"
special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_tokens = [tokenize_en(s) for s in train_data]
en_vocab = vocab.build_vocab_from_iterator(
    en_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

de_tokens = [tokenize_de(s) for s in train_data]
de_vocab = vocab.build_vocab_from_iterator(
    de_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)


In [5]:
def preprocess_and_tokenize(example, en_nlp, de_nlp, max_length, sos_token, eos_token, en_vocab, de_vocab):
    en_tokens = [sos_token] + tokenize_en(example) + [eos_token]
    de_tokens = [sos_token] + tokenize_de(example) + [eos_token]
    en_ids = en_vocab.lookup_indices(en_tokens)
    de_ids = de_vocab.lookup_indices(de_tokens)

    return {"en_tokens": en_tokens, "de_tokens": de_tokens, "en_ids": en_ids, "de_ids": de_ids}


function_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
    "en_vocab": en_vocab,
    "de_vocab": de_vocab
}

train_data = train_data.map(preprocess_and_tokenize, fn_kwargs=function_kwargs)
valid_data = valid_data.map(preprocess_and_tokenize, fn_kwargs=function_kwargs)
test_data = test_data.map(preprocess_and_tokenize, fn_kwargs=function_kwargs)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

kwargs = {
    "type": "torch",
    "columns": ["en_ids", "de_ids"],
    "output_all_columns": True
}

train_data = train_data.with_format(**kwargs)
valid_data = valid_data.with_format(**kwargs)
test_data = test_data.with_format(**kwargs)

In [7]:
def create_collate_function(pad_value):
    def collate_batch(batch):
        english_ids = [example["en_ids"] for example in batch]
        german_ids = [example["de_ids"] for example in batch]

        padded_english_ids = nn.utils.rnn.pad_sequence(english_ids, padding_value=pad_value)
        padded_german_ids = nn.utils.rnn.pad_sequence(german_ids, padding_value=pad_value)

        return {
            "en_ids": padded_english_ids,
            "de_ids": padded_german_ids,
        }

    return collate_batch

def create_data_loader(dataset, batch_size, pad_value, shuffle = False):
    collate_function = create_collate_function(pad_value)
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_function,
        shuffle=shuffle,
    )
    return data_loader


batch_size = 128
pad_value = 0

train_data_loader = create_data_loader(train_data, batch_size, pad_value, shuffle=True)
valid_data_loader = create_data_loader(valid_data, batch_size, pad_value)
test_data_loader = create_data_loader(test_data, batch_size, pad_value)

# Modeling

## 논문 속 모델 설정

- **모델 설정**:  
  - **층 수**: 깊은 LSTM (deep LSTM) 4층  
  - **셀 수**: 각 층당 1000개 셀  
  - **단어 임베딩 차원**: 1000 차원  
  - **입력 어휘 크기**: 160,000  
  - **출력 어휘 크기**: 80,000  
  - **문장 표현**: 8000 실수  
  - **파라미터 개수**: 총 384M 파라미터 (그 중 64M는 순수 재발 연결)  
- **파라미터 초기화**: -0.08과 0.08 사이의 균등 분포  
- **학습 방법**:  
  - **Stochastic Gradient Descent (SGD)**: 모멘텀 없이 고정된 학습률 0.7  
  - **학습률 감소**: 5 에포크 후 매 반 에포크마다 절반으로 감소  
  - **총 학습 에포크**: 7.5 에포크  
- **미니배치 설정**:  
  - 배치 크기: 128 시퀀스  
- **폭발적 그래디언트 방지**:  
  - 그래디언트 크기 제한 (Gradient Clipping)  
  - 각 학습 배치에 대해, $s = \|g\|_2$ (g는 128로 나눈 그래디언트)  
  - 만약 $s > 5$, $ g = \frac{5g}{s}$  
- **다른 문장 길이를 처리하는 방법**:  
  - 짧은 문장 (길이 20-30)과 긴 문장 (길이 > 100)의 균형을 맞추기 위해  
  - 미니배치 내에서 문장 길이를 비슷하게 맞춤  
  - 2배 속도 향상

## 입력 순서 반전의 장점

1. **짧은 단기 의존성 도입:**
   - 원래 문장 순서에서는 소스 문장과 타겟 문장의 대응하는 단어들이 멀리 떨어져 있을 수 있습니다. 예를 들어, 원 문장에서 문장의 처음 부분에 있는 단어는 타겟 문장에서도 처음 부분에 올 가능성이 큽니다. 하지만, 그 사이의 다른 단어들 때문에 학습 모델은 이러한 대응을 찾기 어렵습니다.
   - 입력 문장의 순서를 반전하면 소스 문장의 끝 부분에 있는 단어들이 타겟 문장의 처음 부분에 대응되고, 이로 인해 대응 단어들이 더 가까워집니다. 예를 들어, "a, b, c"를 "α, β, γ"로 매핑할 때 "a"와 "α"는 멀리 떨어진 반면, 순서를 반전시킨 "c, b, a"에서는 "a"와 "α"가 더 가깝게 됩니다.

2. **최소 시간 지연 감소:**
   - 소스 문장과 타겟 문장 사이의 최소 시간 지연(minimal time lag)을 줄입니다. 원래 순서에서는 소스 문장의 각 단어가 타겟 단어와 멀리 떨어져 있어 긴 시간 지연을 가지게 됩니다.
   - 순서를 반전하면, 초기 몇 개의 단어들이 더 가까워져, 최소 시간 지연이 크게 줄어듭니다. 이렇게 되면 역전파(backpropagation) 과정에서 소스와 타겟 사이의 "커뮤니케이션"을 설정하기가 쉬워집니다.

3. **확률 분포의 초반 부분 개선:**
   - 입력 문장을 반전하면 초반 부분의 예측 정확도가 높아지게 됩니다. 이는 학습 초기 단계에서 신경망이 소스 단어와 타겟 단어 사이의 직접적인 관계를 더 잘 학습할 수 있음을 의미합니다.

결과적으로, 이러한 방법은 소스 문장의 정보가 LSTM 네트워크 내에서 더 효과적으로 전달되고 사용되게 하여, 전반적인 번역 성능과 모델 효율성을 크게 향상시킵니다.
이 기술적 변형이 LSTM의 기초 설계의 시퀀스 의존성 문제를 직접적으로 해결하기 때문에 학습이 더 용이해지며, 이는 특히 긴 문장 처리에 있어 큰 이점을 제공합니다.

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_seq) -> tuple:
        embedded = self.dropout(self.embedding(input_seq))
        outputs, (hidden_state, cell_state) = self.rnn(embedded)
        return hidden_state, cell_state

In [9]:
class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate):
        super(Decoder, self).__init__()
        self.output_dim = output_vocab_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout_rate)
        self.fc_out = nn.Linear(hidden_dim, output_vocab_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_token, hidden_state, cell_state):
        input_token = input_token.unsqueeze(0)
        embedded = self.dropout(self.embedding(input_token))
        output, (hidden_state, cell_state) = self.rnn(embedded, (hidden_state, cell_state))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden_state, cell_state

In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size,
                 embedding_dim, hidden_dim, n_layers, dropout_rate, device):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_vocab_size,
                               embedding_dim, hidden_dim, n_layers,
                               dropout_rate)
        self.decoder = Decoder(output_vocab_size,
                               embedding_dim, hidden_dim, n_layers,
                               dropout_rate)
        self.device = device

    def forward(self, src_seq, trg_seq, teacher_forcing_ratio = 0.5):
        batch_size = trg_seq.shape[1]
        trg_len = trg_seq.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden_state, cell_state = self.encoder(src_seq)
        input_token = trg_seq[0, :]

        for t in range(1, trg_len):
            output, hidden_state, cell_state = self.decoder(input_token, hidden_state, cell_state)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input_token = trg_seq[t] if teacher_force else top1

        return outputs

In [11]:
def initialize_weights(model):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [12]:
input_vocab_size = len(de_vocab)
output_vocab_size = len(en_vocab)
embedding_dim = 256
hidden_dim = 512
n_layers = 2
dropout_rate = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seq2seq_model = Seq2Seq(input_vocab_size, output_vocab_size,
                        embedding_dim, hidden_dim, n_layers,
                        dropout_rate, device)
seq2seq_model.to(device)
seq2seq_model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

# Training

In [13]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)
optimizer = optim.Adam(seq2seq_model.parameters())
num_epochs = 10

for epoch in range(num_epochs):
    iterator = tqdm.tqdm(train_data_loader)
    seq2seq_model.train()
    for batch in iterator:
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        optimizer.zero_grad()
        output = seq2seq_model(src, trg, 0.5)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        iterator.set_description(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")


Epoch 10/10 - Loss: 1.9717: 100%|██████████| 227/227 [00:28<00:00,  8.01it/s]


# Testing

In [14]:
seq2seq_model.eval()
with torch.no_grad():
    epoch_loss = 0
    for batch in test_data_loader:
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        output = seq2seq_model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        epoch_loss += loss.item()

print(f'Test Loss: {epoch_loss / len(test_data_loader)}')


Test Loss: 2.1198501139879227
